In [1]:
import numpy as np
import pandas as pd
from numpy import linalg as la
import math
np.random.seed(0)
def multiDimensionalGaussian(rowVector,kthCovrianceMatrix,kthmeanMatrix):
    detKthCovraince=la.det(kthCovrianceMatrix)
    firstVal=math.pow(1/detKthCovraince,0.5)*(1/math.pow((2*math.pi),rowVector.shape[0]/2))
    currMean=rowVector-kthmeanMatrix
    currMeanTranspose=currMean.transpose();
    firstProd=np.dot(currMeanTranspose,la.inv(kthCovrianceMatrix))
    secondProd=np.dot(firstProd,currMean)
    secondProd=secondProd*(-0.5)
    secondVal=math.exp(secondProd)
    return firstVal*secondVal
    
    
def calRic(globalCovarianceMatrix,meanMatrix,dataMatrix,ampList):
   
    ricMatUpdate=np.ones((len(globalCovarianceMatrix),dataMatrix.shape[0]))
    for i in range(0,meanMatrix.shape[0]):
        kthCovrianceMatrix=globalCovarianceMatrix[i]
        kthmeanMatrix=meanMatrix[i].reshape(meanMatrix.shape[1],-1)
        kthamp=ampList[i]
        for k in range(0,dataMatrix.shape[0]):
            ricMatUpdate[i][k]=kthamp*multiDimensionalGaussian(dataMatrix[k].reshape(dataMatrix.shape[1],-1),kthCovrianceMatrix,kthmeanMatrix)
    ricMatUpdate=ricMatUpdate/ricMatUpdate.sum(axis=0,keepdims=1)
    return ricMatUpdate
    
                
def calGaussian(ricMat,dataMatrix):
    global meanMatrix
    global globalCovarianceMatrix
    global ampList
    meanMatrix=np.dot(ricMat,dataMatrix)/ricMat.sum(axis=1,keepdims=1)#mulyiply points and weights,dimenion=3*2
    for i in range(0,meanMatrix.shape[0]):
        currMean=meanMatrix[i]
        calMatrix=dataMatrix-currMean
        curCovarianceMatrix = np.zeros((dataMatrix.shape[1],dataMatrix.shape[1]))
        for j in range(0,ricMat.shape[1]):
            colVector= calMatrix[j].reshape(dataMatrix.shape[1],-1)#convert (d,) to (d,1)
            rowVector=colVector.transpose()
            jthCovariance = np.dot(colVector,rowVector)
            jthCovariance = jthCovariance * ricMat[i][j]
            curCovarianceMatrix = curCovarianceMatrix + jthCovariance
        curCovarianceMatrix=curCovarianceMatrix/np.sum(ricMat[i,:])    
        globalCovarianceMatrix.append(curCovarianceMatrix)
    for i in range(0,ricMat.shape[0]):#calculating amp list
        ampList.append(np.sum(ricMat[i,:])/ricMat.shape[1])            

########               
dataMatrix=np.loadtxt("clusters.txt",delimiter=",")
ricMat=np.random.rand(3,dataMatrix.shape[0])
ricMat=ricMat/ricMat.sum(axis=0,keepdims=1)
ricMatOld=ricMat.copy()
####################################
e=0
while(1):
    globalCovarianceMatrix = []
    ampList=[]
    meanMatrix=np.zeros([3,2])
    calGaussian(ricMat,dataMatrix)
    ricMat=calRic(globalCovarianceMatrix,meanMatrix,dataMatrix,ampList)
    if(np.allclose(ricMat,ricMatOld,atol=0.000001)):
        break;
    ricMatOld=ricMat.copy()
    e=e+1
print("The Amplitudes are ")
print(ampList)
print(" ");
print("The Covariances are")
print(globalCovarianceMatrix);
print(" ");
print("The Means Of Gaussians are")
print(meanMatrix);
print(" ");
print("The membership of each points are")
print(ricMat);


The Amplitudes are 
[0.42812152561753342, 0.12968339869367071, 0.4421950756887959]
 
The Covariances are
[array([[ 3.44042676,  2.29942874],
       [ 2.29942874,  5.17207391]]), array([[ 1.60473961,  0.74376723],
       [ 0.74376723,  1.74864859]]), array([[ 1.05767885,  0.13953817],
       [ 0.13953817,  1.02262812]])]
 
The Means Of Gaussians are
[[ 4.45405331  3.35431221]
 [-1.3448081   1.38041019]
 [-0.8452124  -1.12261945]]
 
The membership of each points are
[[  3.33801761e-03   4.02018540e-03   7.92943640e-03   1.34828235e-03
    1.94113226e-02   9.91829542e-01   2.50901939e-02   5.41254133e-01
    3.19110866e-02   9.96209306e-01   3.07407373e-02   2.87035123e-01
    9.99999899e-01   9.93786690e-01   1.27386390e-03   9.13478918e-01
    1.15293999e-03   2.43901943e-02   1.00080195e-01   9.09515780e-01
    3.76881947e-03   3.74844923e-01   9.99550422e-01   2.29590445e-04
    8.20801284e-04   9.95714862e-01   1.85103848e-02   1.49917727e-02
    8.09318507e-01   9.99999504e-01   1.3

In [2]:
ricMat.transpose()

array([[  3.01548106e-03,   8.66631237e-03,   9.88318207e-01],
       [  3.38210830e-03,   1.48437895e-02,   9.81774102e-01],
       [  2.56265207e-03,   4.34805630e-02,   9.53956785e-01],
       [  9.84543586e-04,   6.73949285e-02,   9.31620528e-01],
       [  1.98513820e-02,   8.63460831e-03,   9.71514010e-01],
       [  9.48870165e-01,   4.74143982e-02,   3.71543699e-03],
       [  1.05833254e-02,   3.06560028e-01,   6.82856646e-01],
       [  2.29554682e-01,   7.53921208e-01,   1.65241099e-02],
       [  3.60840776e-02,   1.95079048e-03,   9.61965132e-01],
       [  9.60020622e-01,   3.87937041e-02,   1.18567400e-03],
       [  1.20949693e-02,   6.14691150e-02,   9.26435916e-01],
       [  1.60986632e-01,   7.05928045e-01,   1.33085324e-01],
       [  9.99843695e-01,   1.56252861e-04,   5.22003794e-08],
       [  9.52374012e-01,   4.52126708e-02,   2.41331723e-03],
       [  2.58559825e-04,   4.39407592e-01,   5.60333848e-01],
       [  9.36766527e-01,   3.20372638e-03,   6.0029746

NameError: name 'testMat' is not defined